In [1]:
import numpy as np
import tensorflow as tf


D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is depr

ImportError: cannot import name 'export_saved_model' from 'tensorflow.python.keras.saving.saved_model' (D:\Anaconda\lib\site-packages\tensorflow\python\keras\saving\saved_model\__init__.py)

In [ ]:
def create_model(shape):

    model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),activation="relu",input_shape=shape),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(1,1), padding='valid'),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(1,1),strides=(1,1),activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(1,1), padding='valid'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dense(1),
    
    ])
    
    loss_fn =tf.keras.losses.MeanSquaredError()
    model.compile(optimizer="adam",
              loss=loss_fn,
              metrics=['accuracy'])

    return model


In [2]:
def grad_des(y_hat,y,model):
    model.fit(np.array([y_hat]), np.reshape(y,(1,)) ,verbose=0)
def forward_prop(state,model):
    predictions = model(np.array([state])).numpy()
    return predictions

In [3]:
def argmax(q_values):
    return q_values.index(max(q_values))

In [4]:
class Qlearning():
    def __init__(self):
        self.epsilon=None
        self.gamma=None
        self.input_shape=None
        self.num_actions=None
        self.models=None
        self.last_action = None
        self.previous_state = None
        self.epsilon_decay=None
        self.min_epsilon=None
    def initilize(self,info):
        self.epsilon=info["epsilon"]
        self.gamma=info["gamma"]
        self.input_shape=info["input_shape"]
        self.num_actions=info["num_actions"]
        self.epsilon_decay=info["epsilon_decay"]
        self.min_epsilon=info["min_epsilon"]
        self.models=[]
        for i in range(self.num_actions):
            self.models.append(create_model(self.input_shape))
        
    def select_action(self, state):
        chosen_action = None
        action_values=[]
        cant_go=self.Cant_Go()
        for x in self.models:
            action_values.append(forward_prop(state,x))
        if self.last_action==None:
            p=[1/4,1/4,1/4,1/4]
        else :
            p=[1/3,1/3,1/3,1/3]
            p[cant_go]=0
            action_values[cant_go]=float("-inf")
        if np.random.random() > self.epsilon: 
            chosen_action = argmax(action_values)
        else:
            chosen_action = np.random.choice(4,p=p)
        self.epsilon-=self.epsilon_decay
        self.epsilon=max(self.min_epsilon,self.epsilon)
        return chosen_action, action_values[chosen_action],max(action_values)
    def start(self,state):
        current_action , self.lastActionValue,maxActionValue= self.select_action(state)
        self.last_action = current_action
        self.previous_state = np.copy(state)
        return self.last_action
    def step(self,reward,state):
        current_action,action_value,maxActionValue=self.select_action(state)
        y=reward+self.gamma*maxActionValue
        grad_des(self.previous_state,y,self.models[self.last_action])
        self.last_action=current_action
        self.previous_state=state
        return self.last_action
    def end(self,reward,state):
        y=reward
        grad_des(self.previous_state,y,self.models[self.last_action])
    def Cant_Go(self):
        if self.last_action==0:
            return 1
        if self.last_action==1:
            return 0
        if self.last_action==2:
            return 3
        if self.last_action==3:
            return 2

NameError: name 'tf' is not defined